In [1]:
import pandas as pd
import numpy as np
# import cPickle 
import _pickle as cPickle
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()
import html.parser as html_parser

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [2]:
TEST_CASE = {}

In [3]:
import re
# import HTMLParser
# html_parser = HTMLParser.HTMLParser()

import html.parser as html_parser

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

nltk.download("wordnet", "/Users/bharath/sourcecode/cynor/skill2vec/nltk_data/")
nltk.data.path.append('./nltk_data/')


import sys
if sys.version_info[0] >= 3:
    unicode = str

def skill_transform(skill, remove_stopwords = True):
    skill = unicode(skill)
    skill = html_parser.unescape(skill)
    
    skill = skill.replace("_", " ").split()
    skill = " ".join([sk for sk in skill if sk])
    
    skill = re.sub(r"\(.*\)", "", skill)
    skill = skill.replace("-", "") \
        .replace(".", "") \
        .replace(",", "") \
        .replace("-", "") \
        .replace(":", "") \
        .replace("(", "") \
        .replace(")", "") \
        .replace(u"åá", "") \
        .replace(u"&", "and") \
        .replace(" js", "js") \
        .replace("-js", "js") \
        .replace("_js", "js") \
        .replace("java script", "js") 
    
    skill = skill.lower()
    
    # Special cases replace
    special_case = {}
    special_case["javascript"] = [ "js", "java script", "javascripts", "java scrip" ]
    special_case["wireframe"] = [ "wireframes", "wire frame", "wire frames", "wire-frame", "wirefram", "wire fram", "wireframing" ]
    special_case["OOP"] = [  "object oriented", "object oriented programming", ]
    special_case["OOD"] = [ "object oriented design", ]
    special_case["OLAP"] = [ "online analytical processing",  ]
    special_case["Ecommerce"] = [ "e commerce",  ]
    special_case["consultant"] = [ "consulting",  ]
    special_case["ux"] = [ "user experience", "web user experience design", "user experience design", "ux designer", "user experience/ux" ]
    special_case["html5"] = [ "html 5",  ]
    special_case["j2ee"] = [ "jee",  ]
    special_case["osx"] = [ "mac os x", "os x" ]
    special_case["senior"] = [ "sr" ]
    special_case["qa"] = [ "quality",  ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["webservice"] = [ "webservices", "website", "webapps" ]
    special_case["xml"] = [ "xml file", "xml schemas", "xml/json", "xml web service" ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["nlp"] = [ "natural language process", "natural language", "nltk" ]
    special_case["aws"] = [ "amazon web service", ]
    
    for root_skill in special_case:
        if skill in special_case[root_skill]:
            skill = root_skill
    
    # Special case regex
    special_case_regex = {
        r'^angular.*$': 'angularjs',
        r'^node.*$': 'nodejs',
        r'^(.*)[_\s]js$': '\\1js',
        r'^(.*) js$': '\\1js',
        r'^(.*) (and|or).*$': '\\1',
    }
    for regex_rule in special_case_regex:
        after_skill = re.sub(regex_rule, special_case_regex[regex_rule], skill)
        if after_skill != skill:
            skill = after_skill
            break
    
    # Stem
    if len(skill) > 2:
        skill_after = skill.split(" ")
        skill_after = [wordnet_lemmatizer.lemmatize(sk, pos="v") for sk in skill_after]
        skill_after = " ".join(skill_after)
        skill = skill_after
    
    # skill stopwords 
    if remove_stopwords:
        skill_stopwords = [ "app", "touch", "the", "application", "programming", "program", "design"
                           "developer", "framework", "development", "programmer", "technologies",
                          "advance", "core"]
        skill_after = skill.split(" ")
        skill = " ".join([ sk for sk in skill_after if sk not in skill_stopwords ])
    
    skill = skill.lower().strip().replace(" ", "_")
    skill = re.sub(' +',' ', skill)
    
    # NOTE: replace js tail
    skill = re.sub('js$','', skill)
    
    return skill

skill_transform("js")
skill_transform("angularjs")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bharath/sourcecode/cynor/skill2vec/nltk_data/..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


'angular'

In [4]:
skill_transform("html5_and_css3_or_html6")

'html5_and_css3'

In [5]:
def test_case(model, model_name):
    cases = [ "machine learning", "js", "javascript", "python", "html", "html5", "css", "angular", "nodejs" ]
    TEST_CASE[model_name] = []
    for case in cases:
        try:
            TEST_CASE[model_name].append(model.similar_by_word(skill_transform(case)))
        except:
            pass

In [6]:
# data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
data_must_have = pd.read_csv('formatted_skill2vec.csv', header = 0, encoding='ISO-8859-1', low_memory=False)
data_must_have.drop_duplicates(inplace=True, keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
del data_must_have["job_brief_id"]
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)

unique_skill_before = len(data_must_have['keyword_name'].unique())
print("Before: %s" % unique_skill_before)

data_must_have['skill'] = data_must_have['keyword_name'].apply(skill_transform)

unique_skill_after = len(data_must_have['skill'].unique())
print("After: %s" % unique_skill_after)

print("Resize: %s%%" % (unique_skill_after * 100.0 / unique_skill_before))

# data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
# data_must_have

Before: 14751
After: 11689
Resize: 79.24208528235374%


In [7]:
# pd.set_option('display.max_rows', 1500)
# data_must_have.drop_duplicates(subset=["keyword_name", "skill"])[["keyword_name", "skill"]].sort_values("skill")[3000:]
musthave_data = data_must_have.groupby('job_title')['skill'].apply(list).tolist()

print ("Training model...")
musthave_model = word2vec.Word2Vec(musthave_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_model.init_sims(replace=True)
test_case(musthave_model, "musthave_model")

2021-10-01 18:21:27,260 : INFO : collecting all words and their counts
2021-10-01 18:21:27,261 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 18:21:27,284 : INFO : PROGRESS: at sentence #10000, processed 45162 words, keeping 7318 word types
2021-10-01 18:21:27,309 : INFO : PROGRESS: at sentence #20000, processed 85922 words, keeping 8578 word types
2021-10-01 18:21:27,347 : INFO : PROGRESS: at sentence #30000, processed 138093 words, keeping 10172 word types
2021-10-01 18:21:27,376 : INFO : PROGRESS: at sentence #40000, processed 188572 words, keeping 11283 word types
2021-10-01 18:21:27,403 : INFO : PROGRESS: at sentence #50000, processed 236926 words, keeping 11516 word types
2021-10-01 18:21:27,432 : INFO : PROGRESS: at sentence #60000, processed 287438 words, keeping 11660 word types
2021-10-01 18:21:27,441 : INFO : collected 11689 word types from a corpus of 303247 raw words and 63130 sentences
2021-10-01 18:21:27,442 : INFO : Creating a fres

Training model...


2021-10-01 18:21:27,545 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 11689 unique words (100.0%% of original 11689, drops 0)', 'datetime': '2021-10-01T18:21:27.516420', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 18:21:27,550 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 303247 word corpus (100.0%% of original 303247, drops 0)', 'datetime': '2021-10-01T18:21:27.549973', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 18:21:27,658 : INFO : deleting the raw counts dictionary of 11689 items
2021-10-01 18:21:27,660 : INFO : sample=0.001 downsamples 16 most-common words
2021-10-01 18:21:27,661 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 297638.3987205968 word co

In [8]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
dice_data = [ [ skill_transform(d.replace("_", " ")) for d in dice if d is not None ] for dice in data_dice if dice is not None ]


musthave_dice_data = dice_data + musthave_data
musthave_dice_data = [ list(set([ d for d in data if d is not None and len(d) > 0 ])) for data in musthave_dice_data  ]

print(len(musthave_dice_data))
musthave_dice_data

print ("Training model...")
musthave_dice_model = word2vec.Word2Vec(musthave_dice_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_model.init_sims(replace=True)
test_case(musthave_dice_model, "musthave_dice_model")

2021-10-01 18:21:41,740 : INFO : collecting all words and their counts
2021-10-01 18:21:41,740 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 18:21:41,828 : INFO : PROGRESS: at sentence #10000, processed 194489 words, keeping 10821 word types
2021-10-01 18:21:41,848 : INFO : PROGRESS: at sentence #20000, processed 236575 words, keeping 13977 word types
2021-10-01 18:21:41,875 : INFO : PROGRESS: at sentence #30000, processed 281633 words, keeping 15034 word types
2021-10-01 18:21:41,906 : INFO : PROGRESS: at sentence #40000, processed 331565 words, keeping 15898 word types
2021-10-01 18:21:41,937 : INFO : PROGRESS: at sentence #50000, processed 377817 words, keeping 16267 word types


70276
Training model...


2021-10-01 18:21:41,967 : INFO : PROGRESS: at sentence #60000, processed 426682 words, keeping 16427 word types
2021-10-01 18:21:42,001 : INFO : PROGRESS: at sentence #70000, processed 475353 words, keeping 16525 word types
2021-10-01 18:21:42,004 : INFO : collected 16525 word types from a corpus of 476908 raw words and 70276 sentences
2021-10-01 18:21:42,006 : INFO : Creating a fresh vocabulary
2021-10-01 18:21:42,094 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 16525 unique words (100.0%% of original 16525, drops 0)', 'datetime': '2021-10-01T18:21:42.094375', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-10-01 18:21:42,095 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 476908 word corpus (100.0%% of original 476908, drops 0)', 'datetime': '2021-10-01T18:21:42.095322', 'gensim': '4.1.2', 'python': '3.8.11 (default,

In [9]:
# model.similar_by_word(skill_transform('machine learning'))

In [10]:
# model.similar_by_word('python')

In [11]:
musthave_dice_naruki_data = musthave_dice_data
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')

data_naruki['skill'] = data_naruki['skill'].apply(skill_transform)
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print(len(data_naruki_final))

for skills in data_naruki_final:
    if len(skills) > 2 and skills not in musthave_dice_naruki_data:
        musthave_dice_naruki_data.append(skills)
        
len(musthave_dice_naruki_data)

120107


168141

In [12]:
# import cPickle as pickle
import _pickle as pickle

pickle.dump( musthave_dice_naruki_data, open( "cynor_model.p", "wb" ) )

In [13]:
print ("Training model...")
musthave_dice_naruki_model = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=300, 
          min_count=3, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

musthave_dice_naruki_model.init_sims(replace=True)
test_case(musthave_dice_naruki_model, "musthave_dice_naruki_model")

2021-10-01 18:27:57,615 : INFO : collecting all words and their counts
2021-10-01 18:27:57,617 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 18:27:57,696 : INFO : PROGRESS: at sentence #10000, processed 194489 words, keeping 10821 word types
2021-10-01 18:27:57,716 : INFO : PROGRESS: at sentence #20000, processed 236575 words, keeping 13977 word types
2021-10-01 18:27:57,742 : INFO : PROGRESS: at sentence #30000, processed 281633 words, keeping 15034 word types
2021-10-01 18:27:57,774 : INFO : PROGRESS: at sentence #40000, processed 331565 words, keeping 15898 word types
2021-10-01 18:27:57,804 : INFO : PROGRESS: at sentence #50000, processed 377817 words, keeping 16267 word types


Training model...


2021-10-01 18:27:57,838 : INFO : PROGRESS: at sentence #60000, processed 426682 words, keeping 16427 word types
2021-10-01 18:27:57,873 : INFO : PROGRESS: at sentence #70000, processed 475353 words, keeping 16525 word types
2021-10-01 18:27:57,916 : INFO : PROGRESS: at sentence #80000, processed 560697 words, keeping 25700 word types
2021-10-01 18:27:57,966 : INFO : PROGRESS: at sentence #90000, processed 647532 words, keeping 33189 word types
2021-10-01 18:27:58,003 : INFO : PROGRESS: at sentence #100000, processed 733997 words, keeping 40411 word types
2021-10-01 18:27:58,045 : INFO : PROGRESS: at sentence #110000, processed 823119 words, keeping 47426 word types
2021-10-01 18:27:58,086 : INFO : PROGRESS: at sentence #120000, processed 909871 words, keeping 52950 word types
2021-10-01 18:27:58,128 : INFO : PROGRESS: at sentence #130000, processed 995489 words, keeping 59091 word types
2021-10-01 18:27:58,167 : INFO : PROGRESS: at sentence #140000, processed 1079903 words, keeping 650

2021-10-01 18:28:23,463 : INFO : Word2Vec lifecycle event {'msg': 'training on 6638465 raw words (6218576 effective words) took 24.5s, 253802 effective words/s', 'datetime': '2021-10-01T18:28:23.463822', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
2021-10-01 18:28:23,464 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=24652, vector_size=100, alpha=0.025)', 'datetime': '2021-10-01T18:28:23.464570', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
/tmp/ipykernel_11421/1428038617.py:12: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrat

In [14]:
TEST_CASE

{'musthave_model': [],
 'musthave_dice_model': [],
 'musthave_dice_naruki_model': []}

In [15]:
TEST_CASE_TABLE = [ pd.DataFrame(TEST_CASE[t]) for t in TEST_CASE ]

In [16]:
[ t for t in TEST_CASE ]

['musthave_model', 'musthave_dice_model', 'musthave_dice_naruki_model']

In [17]:
TEST_CASE_TABLE[0]

""


In [18]:
TEST_CASE_TABLE[1]

""


In [19]:
TEST_CASE_TABLE[2]

""


In [20]:
musthave_dice_naruki_model.wv.save_word2vec_format("musthave_dice_naruki.model")

2021-10-01 18:28:23,648 : INFO : storing 24652x100 projection weights into musthave_dice_naruki.model


In [21]:
!ls -l

total 356528
-rw-r--r--@  1 bharath  staff      6148 Oct  1 16:13 .DS_Store
drwxr-xr-x  13 bharath  staff       416 Oct  1 18:16 .git
-rw-r--r--   1 bharath  staff      1799 Oct  1 15:34 .gitignore
drwxr-xr-x   5 root     staff       160 Oct  1 17:05 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff        11 Oct  1 15:34 README.md
-rw-r--r--   1 root     staff  18301035 Oct  1 18:27 cynor_model.p
-rw-r--r--   1 root     staff  18111038 Oct  1 16:52 cynor_skill2vec_100d.model
-rw-r--r--   1 root     staff  18104991 Oct  1 16:52 cynor_skill2vec_200d.model
-rw-r--r--   1 bharath  staff     67838 Oct  1 18:20 format_data.ipynb
-rw-r--r--   1 root     staff  12376087 Oct  1 18:20 formatted_skill2vec.csv
-rw-r--r--   1 root     staff     13993 Oct  1 18:21 inference.ipynb
-rw-r--r--@  1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  29691555 Oct  1 18:28 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_ful

In [22]:
cynor_skill2vec_model_100d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=100, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

cynor_skill2vec_model_200d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
#           size=200, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
#           iter=4,
          sg =1
    )

2021-10-01 18:28:27,165 : INFO : collecting all words and their counts
2021-10-01 18:28:27,167 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 18:28:27,254 : INFO : PROGRESS: at sentence #10000, processed 194489 words, keeping 10821 word types
2021-10-01 18:28:27,276 : INFO : PROGRESS: at sentence #20000, processed 236575 words, keeping 13977 word types
2021-10-01 18:28:27,307 : INFO : PROGRESS: at sentence #30000, processed 281633 words, keeping 15034 word types
2021-10-01 18:28:27,345 : INFO : PROGRESS: at sentence #40000, processed 331565 words, keeping 15898 word types
2021-10-01 18:28:27,373 : INFO : PROGRESS: at sentence #50000, processed 377817 words, keeping 16267 word types
2021-10-01 18:28:27,408 : INFO : PROGRESS: at sentence #60000, processed 426682 words, keeping 16427 word types
2021-10-01 18:28:27,441 : INFO : PROGRESS: at sentence #70000, processed 475353 words, keeping 16525 word types
2021-10-01 18:28:27,483 : INFO : PROGRESS: at 

2021-10-01 18:28:49,177 : INFO : EPOCH 5 - PROGRESS: at 38.40% examples, 214326 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:28:50,240 : INFO : EPOCH 5 - PROGRESS: at 64.39% examples, 244721 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:28:51,251 : INFO : EPOCH 5 - PROGRESS: at 88.54% examples, 259878 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:28:51,665 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-10-01 18:28:51,676 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-10-01 18:28:51,727 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-10-01 18:28:51,743 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-10-01 18:28:51,743 : INFO : EPOCH - 5 : training on 1327693 raw words (1224830 effective words) took 4.6s, 264591 effective words/s
2021-10-01 18:28:51,744 : INFO : Word2Vec lifecycle event {'msg': 'training on 6638465 raw words (6124158 effective words) took 23.4s, 262260 effective words/s'

2021-10-01 18:29:06,648 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-10-01 18:29:06,677 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-10-01 18:29:06,697 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-10-01 18:29:06,698 : INFO : EPOCH - 3 : training on 1327693 raw words (1224627 effective words) took 4.6s, 267487 effective words/s
2021-10-01 18:29:07,816 : INFO : EPOCH 4 - PROGRESS: at 8.58% examples, 186613 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:29:08,830 : INFO : EPOCH 4 - PROGRESS: at 43.32% examples, 230439 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:29:09,860 : INFO : EPOCH 4 - PROGRESS: at 67.08% examples, 253853 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:29:10,889 : INFO : EPOCH 4 - PROGRESS: at 91.91% examples, 267696 words/s, in_qsize 7, out_qsize 0
2021-10-01 18:29:11,132 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-10-01 18:29:11,144 : INFO : worker thread f

In [23]:
cynor_skill2vec_model_100d.wv.save_word2vec_format("cynor_skill2vec_100d.model")
cynor_skill2vec_model_200d.wv.save_word2vec_format("cynor_skill2vec_200d.model")

2021-10-01 18:29:15,849 : INFO : storing 19227x100 projection weights into cynor_skill2vec_100d.model
2021-10-01 18:29:17,618 : INFO : storing 19227x100 projection weights into cynor_skill2vec_200d.model


In [24]:
!ls -l
!free -m

total 373344
-rw-r--r--@  1 bharath  staff      6148 Oct  1 16:13 .DS_Store
drwxr-xr-x  13 bharath  staff       416 Oct  1 18:16 .git
-rw-r--r--   1 bharath  staff      1799 Oct  1 15:34 .gitignore
drwxr-xr-x   5 root     staff       160 Oct  1 17:05 .ipynb_checkpoints
-rw-r--r--   1 bharath  staff        11 Oct  1 15:34 README.md
-rw-r--r--   1 root     staff  18301035 Oct  1 18:27 cynor_model.p
-rw-r--r--   1 root     staff  22427998 Oct  1 18:29 cynor_skill2vec_100d.model
-rw-r--r--   1 root     staff  22398520 Oct  1 18:29 cynor_skill2vec_200d.model
-rw-r--r--   1 bharath  staff     67838 Oct  1 18:20 format_data.ipynb
-rw-r--r--   1 root     staff  12376087 Oct  1 18:20 formatted_skill2vec.csv
-rw-r--r--   1 root     staff     13993 Oct  1 18:21 inference.ipynb
-rw-r--r--@  1 bharath  staff   8885546 Sep 30 21:56 mustHaveSkills-2.csv
-rw-r--r--   1 root     staff  29691555 Oct  1 18:28 musthave_dice_naruki.model
-rw-r--r--   1 bharath  staff  18240737 Sep 30 21:56 naukri_skill_ful

In [25]:
musthave_dice_naruki_model.wv.similar_by_word(skill_transform('javascript'),topn=10)

[('css3', 0.8287642002105713),
 ('oops_concept', 0.8168147802352905),
 ('ajax', 0.80660080909729),
 ('html5', 0.801425039768219),
 ('xhtml', 0.7971700429916382),
 ('web_ui_developer', 0.7898422479629517),
 ('web_developement', 0.788322925567627),
 ('webdeveloper', 0.7870904207229614),
 ('mvc_frameworks', 0.7866816520690918),
 ('css_cod', 0.786259651184082)]

In [26]:
cynor_skill2vec_model_200d.wv.similar_by_word(skill_transform('gcp'))

[('dq', 0.946877121925354),
 ('mfr', 0.9346513748168945),
 ('bet', 0.9302838444709778),
 ('gamp5', 0.9295294284820557),
 ('regulatory_affairs_executive', 0.9291613101959229),
 ('regulatory_market', 0.9250781536102295),
 ('biosimilar', 0.9230036735534668),
 ('ardl', 0.9225677847862244),
 ('cell_base_assay', 0.9178791046142578),
 ('mab', 0.9166686534881592)]

In [27]:
# [(u'css3', 0.9307963252067566),
#  (u'bootstrap', 0.9234713912010193),
#  (u'ui_developer', 0.9192823171615601),
#  (u'front_end_engineer', 0.9077576398849487),
#  (u'front__end', 0.9058327674865723),
#  (u'css_3', 0.9039251208305359),
#  (u'ui_engineer', 0.9014566540718079),
#  (u'adobe_experience_management_suite', 0.9006956815719604),
#  (u'html4', 0.9003456234931946),
#  (u'user_interface_developer', 0.8981920480728149)]

In [28]:
musthave_dice_naruki_model.similar_by_word(skill_transform('javascript'))

AttributeError: 'Word2Vec' object has no attribute 'similar_by_word'

In [29]:
musthave_dice_naruki_model.similar_by_word(skill_transform('node_js'), 300)

AttributeError: 'Word2Vec' object has no attribute 'similar_by_word'

In [30]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('node_js'), 300)

NameError: name 'musthave_dice_naruki_model_100d' is not defined

In [ ]:
re.sub(r'^angular.*$', 'angular', "angular.js")